In [93]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Program, Task
import pandas as pd
from dataclasses import replace

from code_widget.example import CodeWidget
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
# CHANGE ME!
TASK_ID = 'reachable'
AUTHOR = 'scott'

In [96]:
task = Task(
    id=TASK_ID,
    name="reflexive-transitive closure",
    description="Given a graph and a vertex v, list the vertices reachable from v (including v)",
    category="Graph Reachability",
    plan=[{
        "id": "necessary",
        "description": "compute (subset of) reachability relation",
    }, {
        "id": "source",
        "description": "select only vertices reachable from v",
    }
    ],
    sample_input={
        "graph": [
            {"source": "a", "target": "b"},
            {"source": "b", "target": "b"},
            {"source": "b", "target": "c"},
            {"source": "c", "target": "f"},
            {"source": "c", "target": "b"},
            {"source": "a", "target": "d"},
            {"source": "e", "target": "a"},
        ],
        "query": [{"source": "a"}]
    },
    sample_output=["a", "b", "c", "d", "f"],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [59]:
datalog = replace(prototype,
    language='datalog',
    source='''.decl path(x: symbol, y: symbol)
path(x, y) :- graph(x, y).
path(x, y) :- graph(x, z), path(z, y).
reachable(x) :- query(source), path(source, x).''').load_plan()
datalog.execute(task)
datalog.save()

In [60]:
sql = replace(prototype,
    language='sql',
    source='''WITH RECURSIVE
closure(source, target) AS (
  SELECT DISTINCT source, target FROM graph
  UNION
  SELECT edge.source, path.target
  FROM closure as path JOIN graph as edge
  ON edge.target = path.source
)
SELECT S.target FROM closure as S
JOIN query ON S.source = query.source''').load_plan()
sql.execute(task)
sql.save()

In [92]:
python_imp = replace(prototype,
    language='python-imperative',
    implementation='',
    source='''def reachable(graph, query):
    adjacency_list = defaultdict(list)
    for edge in graph:
        adjacency_list[edge["source"]].append(edge["target"])
    source = query[0]["source"]
    
    visited = set()
    to_visit = set([source])
    
    while len(to_visit) > 0:
        current = to_visit.pop()
        if current in visited:
            continue
        for neighbor in adjacency_list[current]:
            to_visit.add(neighbor)
        visited.add(current)
            
    return list(visited)''')
python_imp.execute(task)
python_imp.save()

In [81]:
# python_fun = replace(prototype,
#     language='python-functional',
#     implementation='',
#     source='''def reachable(graph, query):
#     def step(relation):
#         return set([
#             (source, edge["target"])
#             for (source, target) in relation
#             for edge in graph
#             if target == edge["source"]
#         ]).union(relation)

#     def fix(f, x):
#         next = f(x)
#         return x if next == x else fix(f, next)

#     source = query[0]["source"]
#     initial = set([(source, source)])
#     return list(set([v for (_, v) in fix(step, initial)]))''')
# python_fun.execute(task)
# python_fun.save()

In [85]:
python_fun = replace(prototype,
    language='python-functional',
    implementation='',
    source='''def reachable(graph, query):
    def step(visited):
        return set([
            edge["target"]
            for vertex in visited
            for edge in graph
            if vertex == edge["source"]
        ]).union(visited)

    def fix(f, x):
        next = f(x)
        return x if next == x else fix(f, next)

    source = query[0]["source"]
    return list(fix(step, set([source])))''')
python_fun.execute(task)
python_fun.save()

In [64]:
# todo
pandas = replace(prototype,
    language='python-pandas',
    implementation='',
    source='''def scc(graph):
    
''')
pandas.execute(task)
pandas.save()

SyntaxError: unexpected EOF while parsing (<string>, line 6)